# Objectives
## Research Questions to Answer: 
### 1. What are the final rankings and scores for the season?
### 2. What is the most ideal candidate for current players?
### 3. Which statistics actually impact performance or outcome of games?
####      a. How do you value one position vs. another in fantasy?
####      b. Does the volume the yards thrown or ran count for an ideal candidate?
### 4. Potential Revenue based on contracts (and social media)


In [6]:
#Dependencies
import os
import urllib
import json
import simplejson
import csv
import pandas as pd
import pytest
import base64
import requests
from pprint import pprint
from datetime import datetime
from ohmysportsfeedspy.v1_0 import API_v1_0
from ohmysportsfeedspy.v1_1 import API_v1_1
from ohmysportsfeedspy.v1_2 import API_v1_2
from ohmysportsfeedspy import MySportsFeeds
msf = MySportsFeeds(version="1.2")

In [7]:
#Authentication
msf.authenticate("mdiaz","EC1000ltd")

# NFL Schedule Function
#### (Trying to turn this into 1 api call function)

In [8]:
#Build a url path:
#Create a dictionary of information needed/intested in
#Open Necessary Files
username = "mdiaz"
password = "EC1000ltd"
baseUrl = "https://api.mysportsfeeds.com/v1.2/pull/nfl/"
Authorization = "mdiaz" + ":" + "EC1000ltd"
format_type = [".csv", ".json", ".xml"]
nfl_teams = ["ARI","ATL","BAL","BUF","CAR","CHI","CIN","CLE",
              "DAL","DEN","DET","GB","HOU","IND","JAX","KC",
              "LAC","LAR","MIA","MIN","NE","NO","NYG","NYJ",
              "OAK","PHI","PIT","SD","SEA","SF","STL","TB","TEN","WAS"]
seasons = ["2014-2015-regular/", "2015-playoff/",
         "2015-2016-regular/", "2016-playoff/",
         "2016-2017-regular/", "2017-playoff/",
         "2017-2018-regular/", "2018-playoff/"]
#API ID Parameters:
api_params = ["?gameid=",
              "?fordate="]
#Feed Paramters that DON'T USE fordates/gameid
overall_feed = ["active_players",
        "conference_team_standings"
        "cumulative_player_stats",
        "division_team_standings",
        "full_game_schedule",
        "latest_updates",
        "overall_team_standings",
        "player_injuries",
        "playoff_team_standings"]
#Feed Parameters that use ForDates:
fordate_feed = ["current_season",
                "daily_dfs",
                "daily_game_schedule",
                "daily_player_stats",
                "roster_players",
                "scoreboard",
                "roster_players"]
#Feed Parameters that use GameID:
gameid_feed = ["game_boxscore",
               "game_playbyplay",
               "game_startinglineup"]


headers= {"Authorization": "Basic " + base64.b64encode('{}:{}'.format(username,password).encode('utf-8')).decode('ascii')}
#Building a list to append through nfl_schedule function to parse the dates, away, home team
nfl_season = []
nfl_dates = []
nfl_awayTeam = []
nfl_homeTeam = []

nfl_dateDict = pd.DataFrame({
                            "Season": nfl_season,
                            "Dates": nfl_dates,
                            "Away Team": nfl_awayTeam,
                            "Home Team": nfl_homeTeam})

#NFL Functions for full game, active players, standings, injuries(incomplete), latest updates
# nfl_dateDict
def nfl_schedule():
    
    for season in seasons:
        season_loop = season
        print(season_loop)
        season_l = season_loop   
        url2 = "https://api.mysportsfeeds.com/v1.2/pull/nfl/" + season_l + "full_game_schedule" + format_type[1]
        print(url2)
        nflsch = requests.get(url2, headers= headers)
#         print(nflsch.json())
        x = nflsch.json()
        games_List = x["fullgameschedule"]["gameentry"]
        currentGame = games_List[0]
        pprint(currentGame)
        print(len(games_List))
        
#Trying to print game box scores within schedule
nfl_schedule()

2014-2015-regular/
https://api.mysportsfeeds.com/v1.2/pull/nfl/2014-2015-regular/full_game_schedule.json
{'awayTeam': {'Abbreviation': 'GB',
              'City': 'Green Bay',
              'ID': '62',
              'Name': 'Packers'},
 'date': '2014-09-04',
 'delayedOrPostponedReason': None,
 'homeTeam': {'Abbreviation': 'SEA',
              'City': 'Seattle',
              'ID': '79',
              'Name': 'Seahawks'},
 'id': '30509',
 'location': 'CenturyLink Field',
 'originalDate': None,
 'originalTime': None,
 'scheduleStatus': 'Normal',
 'time': '8:30PM',
 'week': '1'}
256
2015-playoff/
https://api.mysportsfeeds.com/v1.2/pull/nfl/2015-playoff/full_game_schedule.json
{'awayTeam': {'Abbreviation': 'ARI',
              'City': 'Arizona',
              'ID': '76',
              'Name': 'Cardinals'},
 'date': '2015-01-03',
 'delayedOrPostponedReason': None,
 'homeTeam': {'Abbreviation': 'CAR',
              'City': 'Carolina',
              'ID': '69',
              'Name': 'Panthers

In [2]:
#NFL Schedule without Loop for using following functions
j = requests.get("https://api.mysportsfeeds.com/v1.2/pull/nfl/2014-2015-regular/full_game_schedule.json", headers=headers)
print(j)
x =j.json()
games_List = x["fullgameschedule"]["gameentry"]
currentGame = games_List[0]
print(json.dumps(currentGame, indent=4, sort_keys=True))
print(len(games_List))


In [43]:
#Retrieve gameid for boxscores, starting lineups, 
def nfl_schedule():
    
    for season in seasons:
        season_loop = season
        print(season_loop)
        season_l = season_loop   
        url2 = baseUrl + season_l + "full_game_schedule" + format_type[1]
        print(url2)
        nflsch = requests.get(url2, headers= headers)
#         print(nflsch.json())
        x = nflsch.json()
        games_List = x["fullgameschedule"]["gameentry"]
        currentGame = games_List[0]
#         pprint(currentGame)
#         print(len(games_List))
        
#Trying to print game box scores within schedule
    def gameID(games_List, baseUrl):
        for game in games_List:
            dateOfCurrentGame = game["date"]
            dateFormatted = dateOfCurrentGame.replace("-", "")
            awayTeamCurrentGame = game["awayTeam"]["Abbreviation"]
            homeTeamCurrentGame = game["homeTeam"]["Abbreviation"]
            for season in seasons:
                season_loop = season
                print(season_loop)
                season_2 = season_loop  
                dateOfCurrentGame = currentGame["date"]
                dateFormatted = dateOfCurrentGame.replace("-", "")
                awayTeamCurrentGame = currentGame["awayTeam"]["Abbreviation"]
                homeTeamCurrentGame = currentGame["homeTeam"]["Abbreviation"]
                for types in gameid_feed:
                    feed_loop = types
                    print(feed_loop)
                    feed_1 = feed_loop
                    boxScoresURL = baseUrl + season_2 + feed_1 + format_type[1] + api_params[0] + dateFormatted + "-" + awayTeamCurrentGame + "-" + homeTeamCurrentGame
                    print(boxScoresURL)


                    nfl_boxscores = requests.get(boxScoresURL, headers=headers)
#                   print(nfl_boxscores.json())
                    y = nfl_boxscores.json()
                    print(json.dumps(y, indent=4, sort_keys=True))
    #         nfl_boxscores_json = y["gameboxscore"]["game"]
#             print(json.dumps(nfl_boxscores_json, indent=4, sort_keys=True))
    #         game_info = nfl_boxscores_json
    #         print(game_info)
    #         print(len(nfl_boxscores_json))
    gameID(games_List , baseUrl)
nfl_schedule()

NameError: name 'games_List' is not defined

In [4]:
#Retrive Fordates for scoreboard, roster, daily game schedule, roster players, current season,  
def Fordates(games_List, baseUrl):
    for game in games_List:
        datesPerSeason = game["date"]
        dateFormatted = datesPerSeason.replace("-", "")
        for season in seasons:
            season_loop = season
            print(season_loop)
            season_l = season_loop
            for types in fordate_feed:
                feed_loop = types
                print(feed_loop)
                feed_2 = feed_loop
                boxScoresURL = baseUrl + season_l + feed_2 + format_type[1] + api_params[1] + dateFormatted
                
#             boxScoresURL = baseUrl + season_l + "roster_players.json?fordate=" + dateFormatted
                print(boxScoresURL)
                nfl_boxscores = requests.get(boxScoresURL, headers=headers)
#                 print(nfl_boxscores.json())
                y = nfl_boxscores.json()
                print(json.dumps(y, indent=4, sort_keys=True))
    #         nfl_boxscores_json = y["gameboxscore"]["game"]
#             print(json.dumps(nfl_boxscores_json, indent=4, sort_keys=True))
    #         game_info = nfl_boxscores_json
    #         print(game_info)
    #         print(len(nfl_boxscores_json))
Fordates(games_List , baseUrl)

## 1. What are the final rankings and scores for the season?